In [148]:
###-----------------
### Import Libraries
###-----------------

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections.abc import Callable
from typing import Literal

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable

from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

%matplotlib inline

In [149]:
###----------------
### Some parameters
###----------------

inpDir = './DATA/KAGGLE'
outDir = './home/dai/Downloads/Sahil/Final Project/DeepFake Audio Analyzer/KAGGLE/output'
subDir = 'Dataset'
audDir = "./DATA/KAGGLE/AUDIO"

RANDOM_STATE = 24 # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results

EPOCHS = 11 # number of epochs
ALPHA = 0.001 # learning rate
NUM_SAMPLES = 1280 # How many samples we want to generate 
NOISE = 0.2 # Noise to be introduced in the data
TEST_SIZE = 0.2

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

In [150]:
data_df = pd.read_csv(os.path.join(inpDir, 'DATASET-balanced.csv'))
data_df.head()

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.338055,0.027948,2842.948867,4322.916759,6570.586186,0.041050,-462.169586,90.311272,19.073769,24.046888,...,-6.686564,0.902086,-7.251551,-1.198342,4.747403,-4.986279,0.953935,-5.013138,-6.779060,FAKE
1,0.443766,0.037838,2336.129597,3445.777044,3764.949874,0.047730,-409.413422,120.348808,-7.161531,5.114784,...,-2.131157,-6.876417,-1.359395,0.326401,-5.420016,-2.109968,-1.757634,-9.537907,-8.494421,FAKE
2,0.302528,0.056578,2692.988386,2861.133180,4716.610271,0.080342,-318.996033,120.490273,-24.625771,23.891073,...,-5.853725,-3.724773,-6.627182,-5.117002,-6.072106,-0.994653,-1.617120,-3.922354,-7.033001,FAKE
3,0.319933,0.031504,2241.665382,3503.766175,3798.641521,0.047180,-404.636749,136.320908,2.308172,-3.907071,...,-1.898315,-2.046493,-7.176277,-3.293508,4.209121,0.121835,-5.407063,-3.654926,-3.274857,FAKE
4,0.420055,0.016158,2526.069123,3102.659519,5025.077899,0.051905,-410.497925,152.731400,-18.266771,51.993462,...,-1.952340,0.810868,6.238493,6.555839,7.535542,2.849219,2.616843,-1.793357,-5.060998,FAKE


In [151]:
### Imports
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


In [152]:
y, sr = librosa.load("/mnt/5866F9FE66F9DCA6/Puneet CDAC-Practice/CDAC_Project/DATA/KAGGLE/AUDIO/FAKE/Biden/biden-to-linus.wav")


/tmp/ipykernel_16826/1559504763.py:1: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load("/mnt/5866F9FE66F9DCA6/Puneet CDAC-Practice/CDAC_Project/DATA/KAGGLE/AUDIO/FAKE/Biden/biden-to-linus.wav")
/home/sanjiv/anaconda3/envs/project/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/5866F9FE66F9DCA6/Puneet CDAC-Practice/CDAC_Project/DATA/KAGGLE/AUDIO/FAKE/Biden/biden-to-linus.wav'

In [ ]:
len(y) # Length in seconds * Samples per second

13230000

In [ ]:
sr # Sample rate

22050

In [ ]:
round(len(y)/sr) # Returns 10min or 600 secs

600

In [ ]:
def extract_features(file_path, segment_length):   # Function to extract features from an audio file
    
    try:
        
        y, sr = librosa.load(file_path) 
        #  Loading audio files returns 
        # y[audio time series. Multi-channel is supported]
        # sr[sampling rate of y] Note: Taking default 22050
        # [For more details : https://librosa.org/doc/0.10.1/generated/librosa.load.html]

        
        num_segments = int(np.ceil(len(y) / float(segment_length * sr))) 
        # Calculate the number of segments based on the segment length and audio length
        
        
        features = [] 
        # Initialize a list to store the features for this file

        
        for i in range(num_segments): # Extracting features for each segment
            
            start_frame = i * segment_length * sr   # Calculate start for the current segment
            end_frame = min(len(y), (i + 1) * segment_length * sr)    # Calculate  end frame for the current segment
            # making sure the last frame does not excede the lenght of audio time series

            
            y_segment = y[start_frame:end_frame]# Extract audio for current segment of audio file


            # Extract different features
            chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_segment, sr=sr))
            # For more details : https://conference.scipy.org/proceedings/scipy2015/pdfs/brian_mcfee.pdf
            rms = np.mean(librosa.feature.rms(y=y_segment))
            spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_segment, sr=sr))
            spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_segment, sr=sr))
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_segment, sr=sr))
            zcr = np.mean(librosa.feature.zero_crossing_rate(y_segment))
            mfccs = librosa.feature.mfcc(y=y_segment, sr=sr) # n_mfcc=20 by default
            mfccs_mean = np.mean(mfccs, axis=1)
            
            # Append the extracted features to the list
            features.append([chroma_stft, rms, spec_cent, spec_bw, rolloff, zcr, *mfccs_mean])

        return features
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [ ]:
# Function to create the dataset
def create_dataset(audio_dir, segment_length):
    
    labels = ['FAKE', 'REAL'] # Label for y
    feature_list = []

    # Iterate over all files in the audio_dir
    for label in labels:
        print(f'Processing {label} files...')
        files = os.listdir(os.path.join(audio_dir, label))
        # Wrap the files iterable with tqdm to show the progress bar
        for file in files:
            file_path = os.path.join(audio_dir, label, file)
            # Extract features for the current file
            file_features = extract_features(file_path, segment_length)
            if file_features:
                # Append features of all segments along with the label to the dataset
                for segment_features in file_features:
                    feature_list.append(segment_features + [label])
                    
    # Create a DataFrame with the dataset
    df = pd.DataFrame(feature_list, columns=['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20','LABEL'])
    
    return df


In [ ]:
# Create the dataset
dataset = create_dataset(audDir, 1)

Processing FAKE files...
Processing REAL files...


In [ ]:
# Save the dataset to a CSV file
csv_output_path = './DATA/csvs/Test_audio_features_1.csv'
dataset.to_csv(csv_output_path, index=False)

print(f'Dataset created and saved to {csv_output_path}')

Dataset created and saved to ./DATA/csvs/Test_audio_features_1.csv


In [180]:
audio_df = pd.read_csv('./DATA/csvs/Test_audio_features_1.csv')

In [181]:
audio_df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.291005,0.100972,1503.819020,2003.900473,2640.264338,0.058461,-227.28400,117.363870,-21.856697,-5.956208,...,0.231222,-4.286140,-0.658060,-17.671778,0.492580,-8.939298,-10.583409,-9.856175,0.110423,FAKE
1,0.488654,0.005333,2757.999207,2607.776140,5640.965132,0.129239,-376.87490,82.629425,-4.113033,13.557878,...,-1.619721,-5.480615,-1.591572,-9.422898,-4.021746,-7.764452,-3.283262,-6.914111,-3.745474,FAKE
2,0.320947,0.059529,1876.721927,2279.032510,3907.786976,0.085582,-284.79144,104.583750,1.505778,11.288380,...,-4.458688,-9.660798,-4.906862,-12.124472,-6.083214,-8.779160,-4.301162,-9.839105,-2.189336,FAKE
3,0.317934,0.053302,2061.158776,2186.794174,3676.794434,0.099088,-258.28410,94.734474,1.924067,23.870354,...,-7.312897,-9.438956,-7.222197,-11.617744,-1.504134,-6.931713,-4.455357,-3.408775,-0.722356,FAKE
4,0.392725,0.048176,2213.382506,2198.764449,4162.759677,0.132568,-307.62920,100.989970,-14.262594,8.875005,...,-2.009577,-4.004641,-5.763664,-7.626152,-1.623855,-5.220128,-1.803224,-4.053137,2.000329,FAKE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29960,0.380453,0.032703,2666.514179,2112.847586,4668.300559,0.163064,-289.70065,79.301810,-34.634933,-2.610724,...,1.182776,-15.351231,-2.042419,-9.185847,-2.628170,-11.762793,-5.393685,-8.502969,-2.474539,REAL
29961,0.317955,0.036571,2310.894483,2100.396249,4078.339733,0.125388,-264.99075,87.606865,-33.935280,10.865050,...,-5.449326,-13.275034,5.052628,-9.755420,-6.369122,-11.137696,-3.693965,-7.176721,-5.510276,REAL
29962,0.417653,0.041490,1943.153338,1824.786318,3402.001398,0.105247,-205.10078,105.617230,-65.244415,-11.780135,...,-1.176161,-11.554142,-4.132590,-11.246901,0.278845,-11.095166,-3.507075,-4.943163,3.291279,REAL
29963,0.401598,0.037857,2042.405086,1803.789277,3407.874090,0.118852,-200.52167,85.288734,-90.497856,-33.224700,...,4.300089,-3.402192,-4.930651,-13.453515,-5.114316,-7.686623,1.444214,-2.637488,2.451039,REAL


In [182]:
audio_df.describe().T

,count,mean,std,min,25%,50%,75%,max
chroma_stft,29965.0,0.395463,0.090617,0.150902,0.330103,0.388381,0.451087,0.756753
rms,29965.0,0.047151,0.035505,0.000297,0.018399,0.040530,0.067505,0.321696
spectral_centroid,29965.0,2293.167409,694.870931,683.692420,1804.612796,2189.217450,2689.100367,5665.103886
spectral_bandwidth,29965.0,2139.073967,397.395190,755.011186,1873.436407,2138.131932,2413.649229,3608.060490
rolloff,29965.0,4227.765637,1304.156458,955.291193,3268.887052,4066.839045,5042.929355,9128.365257
zero_crossing_rate,29965.0,0.123934,0.057758,0.009277,0.083374,0.111339,0.151955,0.509976
mfcc1,29965.0,-309.819738,87.949068,-695.255300,-362.891050,-294.691860,-242.166700,-114.405830
mfcc2,29965.0,93.464118,33.922410,-44.360690,71.518486,94.985710,115.101510,232.084140
mfcc3,29965.0,-12.380376,26.935078,-128.548230,-25.353941,-7.077191,6.280376,72.277850
mfcc4,29965.0,12.332665,18.265384,-70.541046,1.264601,14.055121,24.804890,91.189200


In [183]:
corr = audio_df.drop(columns='LABEL').corr()

In [184]:
(corr['chroma_stft']<=0.05) & (corr['chroma_stft']>=-0.05)

chroma_stft           False
rms                   False
spectral_centroid     False
spectral_bandwidth    False
rolloff               False
zero_crossing_rate    False
mfcc1                 False
mfcc2                 False
mfcc3                  True
mfcc4                  True
mfcc5                  True
mfcc6                 False
mfcc7                 False
mfcc8                 False
mfcc9                 False
mfcc10                False
mfcc11                False
mfcc12                False
mfcc13                False
mfcc14                False
mfcc15                False
mfcc16                False
mfcc17                False
mfcc18                False
mfcc19                False
mfcc20                False
Name: chroma_stft, dtype: bool

In [185]:
corr.loc[(corr['chroma_stft']<=0.05) & (corr['chroma_stft']>=-0.05)]

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
mfcc3,-0.014020,-0.003616,0.288288,0.510695,0.345786,0.152651,-0.177086,-0.214651,1.000000,0.551826,...,0.037989,-0.017551,-0.007721,-0.016885,0.152948,0.043833,-0.011396,0.051385,-0.065287,-0.051451
mfcc4,-0.026130,0.032911,0.038699,0.090123,0.080620,-0.044088,-0.090875,-0.025824,0.551826,1.000000,...,0.002276,0.144186,0.127429,0.105438,0.276809,0.112934,0.186218,0.052768,0.137918,0.097895
mfcc5,0.009701,0.021405,-0.066477,0.156376,0.003790,-0.132686,-0.157435,0.143798,0.493711,0.618899,...,0.072995,0.056056,0.173012,0.111611,0.256157,0.254312,0.150635,-0.041376,0.188614,0.114872


#### isolation Forest

In [199]:
x = audio_df.drop(columns='LABEL')

esp_range = [0.2,0.4,0.6,1]
mp_range = [2,3,4,5,7,8,9,10,15,20]
scaler = StandardScaler()
scaled = scaler.fit_transform(x)

In [214]:
com = 26
prcomp = PCA(n_components=com)

PC = prcomp.fit_transform(scaled)
col = [f'PC{i}' for i in range(1,com+1)]
PCs = pd.DataFrame(data=PC , columns=col)

In [215]:
PCs

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26
0,-2.869649,-2.323758,-0.774913,-1.013216,-0.758148,-1.907292,0.323708,-1.287913,0.644040,-0.135587,...,-1.128102,1.137607,0.976145,0.127952,-0.000375,0.111174,0.608130,0.391246,-0.196787,0.044003
1,0.931168,2.035948,-0.048948,-0.524140,1.057756,-0.838611,-0.063847,0.174433,-0.166986,-0.688187,...,-0.353848,0.404867,0.419516,-0.532737,-0.154356,0.565728,-0.407825,-0.309874,0.172973,0.052613
2,-2.144672,-0.287611,-1.043627,0.234246,0.414864,-1.304781,0.495062,0.719288,0.368953,-1.073568,...,-0.631701,0.777746,0.526202,-0.344054,-0.102539,0.203245,-0.262750,0.038823,-0.123109,-0.136687
3,-1.460524,-0.348178,-1.419278,0.595289,-0.013579,-1.293163,-0.304809,0.873503,0.178257,-0.484921,...,-0.089055,0.077091,-0.124618,-0.069932,-0.045888,0.094750,-0.407007,0.257007,-0.150807,0.022867
4,0.081600,-0.191262,-0.120798,0.539221,-0.383018,-1.169668,-0.308225,0.328161,0.047682,-0.311122,...,-0.231242,0.001851,-0.326081,0.282416,-0.311766,0.029551,-0.044660,-0.129765,-0.224383,-0.049889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29960,-1.711411,1.247873,0.844540,-1.177802,-0.338025,-0.515884,-1.656085,-0.768093,0.435496,-0.289198,...,0.120525,0.104126,-0.356625,-0.599583,0.863981,-0.261436,0.076646,-0.017868,0.093038,-0.004203
29961,-1.985565,0.372399,-0.321576,-1.327616,-0.400197,0.114311,-1.429839,0.054562,0.359122,0.735304,...,1.250023,0.145772,-0.586536,-1.097577,2.129932,0.375824,-0.518917,0.110692,-0.032536,-0.011080
29962,-1.466422,-1.274894,2.575749,-0.360960,-1.213605,-1.103492,-1.226361,-0.162391,0.433657,-0.234047,...,0.176655,-0.071562,-0.234002,-1.139186,-0.059743,-0.368544,0.047807,0.000085,0.014532,0.007936
29963,-1.139510,-1.463423,4.308652,-0.118576,-2.165539,-0.866741,-1.788724,-0.870678,1.014617,0.938244,...,0.577389,-0.012206,-0.846986,-0.419268,-0.064335,0.475719,-0.221270,0.408859,-0.125674,0.000115


In [216]:
tot_var = np.sum(prcomp.explained_variance_)
cumsum = np.cumsum(prcomp.explained_variance_/tot_var)*100
print("variance percentage of PC scores = ",cumsum) 

variance percentage of PC scores =  [ 24.93089866  44.34369622  54.57496539  60.22835474  65.29981279
  69.29809192  72.60953684  75.5123772   78.14721828  80.53725114
  82.83045662  85.03942327  87.05381339  88.99969985  90.71232047
  92.30485514  93.77409848  95.04089838  96.16080057  97.20277733
  98.1144934   98.94566842  99.60014505  99.89643414  99.97813405
 100.        ]
